You have to work on the files:
*  [Books](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Books.csv.gz)
*  [Book ratings](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Book-Ratings.csv.gz)
*  [Users](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Users.csv.gz)
*  [Goodbooks books](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/goodbooks.csv.gz)
*  [Goodbooks ratings](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/goodbooks-ratings.csv.gz)

### Notes

1.    It is mandatory to use GitHub for developing the project.
1.    The project must be a jupyter notebook.
1.    There is no restriction on the libraries that can be used, nor on the Python version.
1.    To read those files, you need to use the `encoding = 'latin-1'` option.
1.    All questions on the project **must** be asked in a public channel on [Zulip](https://focs.zulipchat.com), otherwise no  answer will be given.

# Progetto di _Foundations of Computer Science_

Camagni Valentina(878252), Grosso Silvia (881993), Merelli Elisa (881427)

In [1]:
#librerie

import pandas as pd
import numpy as np
import re


Importo i dati:

In [ ]:
books = pd.read_csv("https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Books.csv.gz", compression = 'gzip',  encoding = 'latin_1', sep=";", low_memory = False,
                     escapechar= '\\',dtype = {'Year-Of-Publication':'int64'})

#parametri encoding richiesto e compression necessario perchè file .gz da decomprimere
#low_memory = false per warning del codice (var booleana)
#aggiungo dtype come verifica per i campi 

In [ ]:
#aggiungendo l'escape character '\\' in books non ho errore in questo record (errore segnalato grazie a dtype)
books.loc[209538]

In [ ]:
bookrat = pd.read_csv("https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Book-Ratings.csv.gz",compression='gzip', encoding = 'latin-1', sep=";", 
                      dtype = {'User-ID':'int64',
                              'Book-Rating':'int64'})

In [ ]:
users = pd.read_csv("https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Users.csv.gz",compression='gzip', encoding = 'latin-1', sep=";", 
                   dtype = {'User-ID': 'int64',
                            'Age':'float64'})

In [ ]:
gb_books = pd.read_csv("https://github.com/gdv/foundationsCS/raw/master/progetti/2021/goodbooks.csv.gz",compression='gzip', encoding = 'latin-1', sep=",",
                      dtype= {range(1,5) :'int64', 
                              'original_publication_year':'float64',
                              'average_rating':'float64',
                              'ratings_count':'int64', 
                              'work_ratings_count':'int64', 
                              'work_text_reviews_count':'int64',
                              'ratings_1':'int64',
                              'ratings_2':'int64', 
                              'ratings_3':'int64',
                              'ratings_4':'int64', 
                              'ratings_5':'int64'})

In [ ]:
gb_books.head()

In [ ]:
gb_books[['book_id','average_rating']]

In [ ]:
gb_rat = pd.read_csv("https://github.com/gdv/foundationsCS/raw/master/progetti/2021/goodbooks-ratings.csv.gz",compression='gzip', encoding = 'latin-1', sep=",",
                    dtype= {range(1,3):'int64'})

Esploriamo i dati appena caricati, analizzando dimensioni e attributi

In [ ]:
books.shape

In [ ]:
books.head(10)

In [ ]:
bookrat.shape

In [ ]:
bookrat.head(10)

In [ ]:
users.shape

In [ ]:
users.head(10)

In [ ]:
gb_books.shape

In [ ]:
gb_books.head()

In [ ]:
gb_books.columns

In [ ]:
gb_rat.shape

In [ ]:
gb_rat.head(10)

### 1. Normalize the location field of *Users* dataset, splitting into city, region, country.

Abbiamo osservato che in *Location* le stringhe sono separate da virgola e spazio. Utilizzo il metodo split specificando l'espressione regolare di separazione in pat.

In [ ]:
split=users['Location'].str.split(pat = ',\s', expand=True)
split[split[8].isna()==False]

Notiamo che di default vengono create 9 colonne poichè, evidentemente, vi sono records di *users* con più campi relativamente alla *Location*. Decidiamo di definire attraverso una regex il formato *Location* che desideriamo. Gli *users* che non soddisfano tale pattern non verranno considerati

In [ ]:
users=users[users['Location'].str.match("^([a-zA-Z\.\s\/-]+|""),([a-zA-Z\.\s\/-]+|""),([a-zA-Z\.\s\/-]+|"")$")== True].reset_index(drop=True)
users.head()

In [ ]:
users.shape

Splittiamo la colonna Location nei 3 campi *City, Region, Country*, richiesti.

In [ ]:
users[['City','Region','Country']]=users['Location'].str.split(',\s', expand=True)
users.head()

Dopodiché definiamo un nuovo dataframe *users_normalize* dove rimuoviamo la colonna *Location*.

In [ ]:
users_normalize = users.drop('Location', 1) ## ',1' per dire colonna
users_normalize.head()

### 2. For each book in the *Books* dataset, compute its average rating.

Osservando i dataset abbiamo visto che in *bookrat* ci sono più righe riferite allo stesso ISBN perchè corrispondenti alle valutazioni di *'user_id'* differenti. Raggruppiamo quindi per ISBN e calcoliamo la media di rating.

In [ ]:
bookrat.groupby('ISBN').mean()['Book-Rating']

In [ ]:
#verifico calcolo per secondo ISBN 
bookrat[bookrat['ISBN'] == '0375404120']

#qui la media dovrebbe essere 18/7=2.57 -> errore

C'è un errore nel calcolo della media: individuiamo quindi le righe contenenti la stringa '0375404120' nella colonna ISBN con il metodo str.contains

In [ ]:
bookrat[bookrat['ISBN'].str.contains('0375404120')]

Vi sono più righe che contengono questo ISBN rispetto alla ricerca precedente: deduciamo che la presenza di spazi all'inizio e alla fine della stringa ha compromesso i risultati. Infatti verifichiamo che inserendo uno spazio all'inizio della stringa torna il risultato mostrato nella colonna Book-Rating iniziale:

In [ ]:
bookrat[bookrat['ISBN'].str.contains('\s0375404120')]  
#aggiungendo uno spazio davanti ottengo solo più due righe (corrispondo al primo calcolo)

Tolgo quindi gli spazi a capo e coda della stringa ISBN sia in *books* che in *bookrat* con il metodo str.strip

In [ ]:
books.ISBN = books.ISBN.str.strip()

In [ ]:
bookrat.ISBN = bookrat.ISBN.str.strip()

Ora possiamo applicare il metodo groupby sugli ISBN:

In [ ]:
bookrat_avg = bookrat.groupby('ISBN', as_index= False).mean()[['ISBN','Book-Rating']]
bookrat_avg.head()

#as_index= False per togliere ISBN come indice

Effettuiamo ora il merge dei due dataframe sull'ISBN:

In [ ]:
books_bookrat_avg = pd.merge(books,bookrat_avg, on = 'ISBN', how = 'left')
books_bookrat_avg['Book-Rating'] = books_bookrat_avg['Book-Rating'].round(2)   #arrotondo la media a 2 cifre decimali

In [ ]:
books_bookrat_avg[['ISBN','Book-Title', 'Book-Author','Book-Rating']].head(10)

### 3. For each book in the *GoodBooks* dataset, compute its average rating.

In [ ]:
gb_books.columns
#mostro tutte le colonne

Si osserva che è presente la colonna contenente le valutazioni medie per ogni libro:

In [ ]:
gb_books[['book_id','isbn','original_title', 'average_rating']].set_index('book_id')

Controlliamo che i valori nella colonna *'average_rating'* corrispondano alla media pesata del numero di persone che hanno votato 1:5 ('ratings_i') sul numero totale, corrispondente a 'work_ratings_count'.

In [ ]:
#controllo che la somma dei ratings sia sempre corrispondente a work_ratings_count con una condizione booleana
gb_books[(gb_books['work_ratings_count'] == gb_books['ratings_1']+gb_books['ratings_2']+gb_books['ratings_3']+gb_books['ratings_4']+gb_books['ratings_5']) == False].count()

In [ ]:
#verifico ora la corrispondenza dei valori delle medie
gb_books[((((gb_books['ratings_1']*1 + gb_books['ratings_2']*2 + gb_books['ratings_3']*3 + gb_books['ratings_4']*4 + gb_books['ratings_5']*5) / gb_books['work_ratings_count']).round(2) == gb_books['average_rating'])) == False].count()

#effettivamente le medie calcolate manualmente corrispondono alla colonna dell'average

### 4. Merge together all rows sharing the same book title, author and publisher. We will call the resulting datset `merged books`. The books that have not been merged together will not appear in `merged books`.

Innanzitutto effettuiamo un groupby su *'Book-Title','Book-Author','Publisher'*, individuando i conteggi uguali o superiori a 2, corrispondenti ai *merged books*.

In [ ]:
#calcolo numero di righe con stesso book title, author, publisher
merge = books.groupby(['Book-Title','Book-Author','Publisher'], as_index= False).count()
#as_index= False per avere un indice implicito e mantenere le stesse colonne

Prendiamo quindi solo i libri comparsi almeno 2 volte

In [ ]:
merged_books = merge[merge['ISBN'] > 1]
merged_books = merged_books[['Book-Title','Book-Author','Publisher','ISBN']].rename({'ISBN':'Count'}, axis = 1)
#rinomino 'ISBN' con 'count'
merged_books.head(10)

In [ ]:
len(merged_books)
# I libri con stesso titolo, autore e editore sono 4725.

### 5. For each book in `merged books` compute its average rating.

The average is computed considering all books in `books` that have been merged.

Unisco tabelle *books* e *bookrat* con ISBN chiave primaria di books, già corretti precedentemente con strip(), per accertarmi di avere tutto l'elenco di books con valutazioni distinte per isbn e utente (*users-id*)

In [ ]:
books_bookrat = pd.merge(books, bookrat, on = 'ISBN', how = 'left')
books_bookrat.head()

Effettuiamo ora un left join su 'Book-Title','Book-Author','Publisher' tra la tabella *merged_books* e *books_bookrat* ottenuta precedentemente

In [ ]:
merged_books_bookrat = pd.merge(merged_books[['Book-Title','Book-Author','Publisher']], books_bookrat, on = ['Book-Title','Book-Author','Publisher'], how= 'left')
merged_books_bookrat.head()

#ottengo tabella con tutti i books che sono presenti in merged_books

In [ ]:
len(merged_books_bookrat)

Otteniamo un numero maggiore di records. Infatti lo stesso libro individuato da *'Book-Title','Book-Author','Publisher'* può avere edizioni e quindi ISBN differenti e, per ogni edizione, valutazioni da utenti differenti.

Calcolo ora, per ogni libro in *merged_books*, la media di valutazioni, raggruppando anche per ISBN/edizioni differenti.
In questa tabella potremo sapere, per ogni libro che ha avuto più edizioni, la valutazione media di ogni singola edizione.

In [ ]:
merged_books_avg = merged_books_bookrat.groupby(['Book-Title','Book-Author','Publisher','ISBN'], as_index= False).mean().round(2)
merged_books_avg[['Book-Title','Book-Author','Publisher','Book-Rating','ISBN']].rename({'Book-Rating':'Avg-Rating'}, axis = 1)
#abbiamo ottenuto un numero maggiore del numero di libri in merged_books

### 6. For each book in `merged books` compute the minimum and maximum of the average ratings over all corresponding books in the `books` dataset.

Hence for each book in `merged books` we will have exactly two values (a minimum and a maximum)

In [ ]:
dfmin = merged_books_avg.groupby(['Book-Title','Book-Author','Publisher'], as_index = False).min()[['Book-Title','Book-Author','Publisher','Book-Rating']].rename({'Book-Rating':'Min-Rating'}, axis=1)

In [ ]:
seriemax = merged_books_avg.groupby(['Book-Title','Book-Author','Publisher'], as_index = False).max()['Book-Rating']

In [ ]:
pd.concat([dfmin,seriemax], axis = 1).reset_index(drop= True).rename({'Book-Rating':'Max-Rating'}, axis=1)

### 6. For each book in `merged books` compute the minimum and maximum of the average ratings over all corresponding books in the `books` dataset.

Hence for each book in `merged books` we will have exactly two values (a minimum and a maximum)

Calcoliamo la media di rating per ogni edizione (ISBN) di tutti i libri, distinti da *'Book-Title','Book-Author','Publisher'*,  in *books*

In [ ]:
avg_rat_ISBN = books_bookrat.groupby(['Book-Title','Book-Author','Publisher','ISBN'],as_index= False).mean().rename({'Book-Rating':'Avg-Rating'}, axis = 1)
avg_rat_ISBN = avg_rat_ISBN[['Book-Title','Book-Author','Publisher','ISBN','Avg-Rating']].round(2)
avg_rat_ISBN.head()

In [ ]:
merged_books.columns

In [ ]:
avg_rat_ISBN.columns

Creiamo una nuova tabella *extreme_rat* contentente, per ogni libro presente in *merged_books*, l'avg-rating associato a ogni edizione

In [ ]:
extreme_rat = pd.merge(merged_books, avg_rat_ISBN, on = ['Book-Title', 'Book-Author', 'Publisher'], how = 'left')
extreme_rat.head()

Estraiamo quindi, per ogni singolo libro (title,author,publisher), il min e il massimo della media delle valutazioni.

In [ ]:
seriemin = extreme_rat.groupby(['Book-Title', 'Book-Author', 'Publisher']).min().rename({'Avg-Rating':'Min-Rating'}, axis = 1)['Min-Rating']

In [ ]:
seriemax = extreme_rat.groupby(['Book-Title', 'Book-Author', 'Publisher']).max().rename({'Avg-Rating':'Max-Rating'}, axis = 1)['Max-Rating']

Conteggiamo infine il numero di edizioni per ogni libro (title,author,publisher)

In [ ]:
seriecount = extreme_rat.groupby(['Book-Title', 'Book-Author', 'Publisher']).count().rename({'Avg-Rating':'Conteggio'}, axis = 1)['Conteggio']

Otteniamo quindi la tabella definitiva: 
ad ogni libro (title,author,publisher) è associato il minimo e massimo di avg-rating e il numero di edizioni

In [ ]:
pd.concat([seriemin,seriemax, seriecount], axis=1).reset_index()

### 7. For each book in `goodbooks`, compute the list of its authors. Assuming that the number of reviews with a text (column `work_text_reviews_count`) is split equally among all authors, find for each authors the total number of reviews with a text. We will call this quantity the *shared number of reviews with a text*.

In [ ]:
gb_books.head()

In [ ]:
gb_books.columns

In [ ]:
#creo una lista degli autori
lista_autori = gb_books.authors.str.split(', ').tolist()
lista_autori

Creiamo un dizionario che abbia come chiavi i valori contenuti nella colonna *'book_id'* di *gb_books*, e come valore una lista contenente gli autori del libro:

In [ ]:
libri_autori = {}

for i in list(gb_books['book_id']):
    libri_autori[i] = lista_autori[i-1]
                                # [i-1] perchè bookid parte da 1, l'indice implicito parte da 0
    
libri_autori

La seconda parte dell'esercizio richiede di calcolare, per ogni autore il numero totale di reviews with a text, che chiameremo *shared number of reviews with a text*

In [ ]:
#ricavo lista autori singoli ordinati(sorted) e non ripetuti(set) sommando tutte le liste presenti in lista_autori
singoli_autori = sorted(set(sum(lista_autori, [])))
singoli_autori

In [ ]:
#creo la serie rev:
#partendo dai valori nella colonna work_text_reviews_count, divido ogni singolo valore per il numero di autori associati.
#Ottengo così rev come serie di rev partizionate equamente ad ogni libro in gb_books

rev = gb_books['work_text_reviews_count'].copy()
for i in range(len(rev)):
    rev[i] = rev[i]/len(libri_autori[i+1])
                                    #[i+1] perchè rev parte da 0 e libri_autori da 1
rev

Ottenuta così la colonna con il numero di work_text_reviews suddivise per ogni autore, creo un dizionario che associa a ciascun autore lo "shared number of reviews with a text"

In [ ]:
#(libri_autori è il dizionario con liste di autori per ogni book_id)
shared_num_rev = {}
for autore in singoli_autori:
    shared_num_rev[autore] = 0
    for i in libri_autori:
        if autore in libri_autori[i]:
            shared_num_rev[autore] += rev[i-1]

In [ ]:
shared_num_rev

### 8. For each year of publication, determine the author that has the largest value of the shared number of reviews with a text.

In [ ]:
#creo una lista senza ripetizioni di tutti gli anni
years = list(set(gb_books['original_publication_year']))

Creo un dizionario con chiave *original_publication_year* e valori la lista di tutti gli autori dei libri con quell'anno di pubblicazione

In [ ]:
year_authors = {}

for year in years:
    year_authors[year] = []
    for i in gb_books.book_id:
        if gb_books.original_publication_year[i-1] == year:
            year_authors[year] += gb_books.authors[i-1].split(', ')
        
year_authors

Otteniamo un nuovo dizionario: ad ogni chiave corrispondente a *original_pubblication_year* verrà associato l'autore corrispondente con il numero massimo di *shared_num_rev*;  sarà una lista di autori in caso di parimerito. 

In [ ]:
anno_max_rev = {}
for year in year_authors:
    massimo = 0
    anno_max_rev[year] = []
    for author in year_authors[year]:
        if shared_num_rev[author]>= massimo:
            massimo = shared_num_rev[author]
            anno_max_rev[year] += [author]
    
anno_max_rev

### 9. Assuming that there are no errors in the ISBN fields, find the books in both datasets, and compute the difference of average rating according to the ratings and the goodratings datasets

Definiamo con *diff_avg* l'inner join tra il dataset *books_bookrat_avg* (contenente per ogni libro di *books* il rating medio) e *gb_books* (contenente per ogni libro l'*average rating*). La differenza dell'average rating è calcolata dalla semplice differenza dei valori delle due colonne per ogni libro.

In [ ]:
diff_avg = pd.merge(books_bookrat_avg, gb_books, left_on='ISBN', right_on='isbn')
diff_avg['diff-avg'] = diff_avg['Book-Rating'] - diff_avg['average_rating']

In [ ]:
diff_avg[['ISBN','Book-Title','Year-Of-Publication','Publisher','Book-Rating','average_rating','diff-avg']]

### 10. Split the users dataset according to the age. One dataset contains the users with unknown age, one with age 0-14, one with age 15-24, one with age 25-34, and so on.

Costruiamo per ogni fascia d'età un dataframe nuovo ottenuto come filtraggio di *users* in base al range d'età 

Creaimo innanzitutto il DataFrame contenente gli users con età sconosciuta:

In [ ]:
df_unknown = users[users['Age'].isna()].reset_index(drop=True)
df_unknown

Creiamo la lista con i ranges d'età:

In [ ]:
users.Age.max()

In [ ]:
ranges= [-1]+[i for i in range(14,int(users.Age.max())+1,10)]
ranges

Creiamo il dizionario le cui chiavi sono i nomi dei DataFrame distinti per età, e i valori sono i DataFrame contenenti l'insieme dei record degli users filtrati per età associata:

In [ ]:
classi_eta = {}
for i in range(0,len(ranges)-1):
    classi_eta["df_{0}".format(ranges[i+1])] = users[(users['Age']>ranges[i])&(users['Age']<=ranges[i+1])].sort_values('Age').reset_index(drop=True)

In [ ]:
classi_eta

In [ ]:
classi_eta['df_14']

In [ ]:
classi_eta['df_244']

### 11. Find the books that appear only in the goodbooks datasets.

Per ottenere i libri il cui isbn appare solamente nel dataset *gb_books* e non nel dataset *books*, eseguo l'equivalente in pandas del not in in SQL

In [ ]:
gb_books[~ gb_books['isbn'].isin(books['ISBN'])]

### 12. Assuming that each pair (author, title) identifies a book, for each book find the number of times it appears in the books dataset. Which books appear the most times?

Raggruppiamo in *times* i records del dataset *books* con la coppia di attributi *'Book-Title','Book-Author'* e ne contiamo la numerosità

In [ ]:
times = books.groupby(['Book-Title','Book-Author'], as_index = False).count()[['Book-Title','Book-Author','ISBN']].rename({'ISBN':'Count'}, axis = 1)
times.head()

Mostriamo quindi la coppia che appare il maggior numero di volte

In [ ]:
times.iloc[times['Count'].idxmax()]

In [ ]:
times[times['Count'] == times['Count'].max()]

### 13. Find the author with the highest average rating according to the goodbooks datasets.

Creiamo un dizionario che associ ad ogni autore tutti i *book_id* in cui è presente.

In [ ]:
autori_libri = {}

for autore in singoli_autori:
    autori_libri[autore] = []
    for book_id in libri_autori:
        if autore in libri_autori[book_id]:
            autori_libri[autore] += [book_id]      
autori_libri

Associamo ora a ogni autore i valori di *avg_rating* di ogni suo libro

In [ ]:
high_avg_au = {}
for autore in singoli_autori:
    high_avg_au[autore] = []
    for i in autori_libri[autore]:
        high_avg_au[autore] += [gb_books.average_rating[i-1]]
high_avg_au

Concludiamo calcolando per ogni autore la media di *avg_rating* complessiva

In [ ]:
avg_rating = {}
for autore in high_avg_au:
    avg_rating[autore] = np.mean(high_avg_au[autore]).round(2)
avg_rating

Individuiamo l'autore con massimo valore di *avg_rating*

In [ ]:
max_key = max(avg_rating, key=avg_rating. get)
max_key, max(avg_rating.values())